# IMPORT MODULES & FUNCTIONS

In [1]:
#IMPORT PACKAGES
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import pytz #for timezone calculation
import math
import matplotlib.dates as md
import gc
import os
from datetime import datetime
import calendar
import seaborn as sns; sns.set()
import itertools
#import datetime
from time import gmtime, strftime
from matplotlib import cm
from IPython.display import display
#%matplotlib qt
#%matplotlib inline

#SET GLOBAL PARAMETERS
# ================== Global parameters for fonts & sizes =================
FONT_SIZE = 20
rc={'font.size': FONT_SIZE, 'axes.labelsize': FONT_SIZE, 'legend.fontsize': FONT_SIZE, 
    'axes.titlesize': FONT_SIZE, 'xtick.labelsize': FONT_SIZE, 'ytick.labelsize': FONT_SIZE}
plt.rcParams.update(**rc)
plt.rc('font', weight='bold')
 
# For label titles
fontdict={'fontsize': FONT_SIZE, 'fontweight' : 'bold'}
# can add in above dictionary: 'verticalalignment': 'baseline' 

style = 'ggplot' # choose a style from the above options
plt.style.use(style)

from solarcurtailment.energy_calculation import *
from solarcurtailment.clear_sky_day import *
from solarcurtailment.tripping_curt import *
from solarcurtailment.vvar_curt import *
from solarcurtailment.vwatt_curt import *
from solarcurtailment.polyfit import *
from solarcurtailment.file_processing import *
from solarcurtailment.data_visualization import *

#class instantiation
file_processing = FileProcessing()
clear_sky_day = ClearSkyDay()
data_visualization = DataVisualization()
energy_calculation = EnergyCalculation()
tripping_curt = TrippingCurt()
polyfit_f = Polyfit()
vvar_curt = VVarCurt()
vwatt_curt = VWattCurt()

#from FileProcessing 
def input_monthly_files(file_path, data_date_idx):
    """Open time-series D-PV data and ghi data of a certain month. Only compatible for SoLA data format.

    Args:
        file_path (str): The file location of the data
        data_date_idx (str): The month of the files in format 'YYYYMM' eg '201907'

    Returns:
        data (df): the opened & cleaned time-series D-PV data
        ghi (df): the opened & cleaned ghi data
        data_ori (df): the opened & unmodified time-series D-PV data
        ghi_ori (df): the opened & unmodified ghi data
    """
    
    data_path = file_path + r"/processed_unsw_" + data_date_idx + '_data_raw.csv'
    data_ori = pd.read_csv(data_path)
    data = data_ori.set_index('utc_tstamp')

    # Convert timestamp to local Adelaide time
    data.index = pd.to_datetime(data.index) # convert index from object type to datetime
    Adelaide_local_time = pytz.timezone('Australia/Adelaide')
    data.index = data.index.tz_localize(pytz.utc).tz_convert(Adelaide_local_time) # convert utc to local adelaide time
    data.index.rename('Timestamp', inplace = True)

    # Load GHI data
    ghi_date_idx = data_date_idx[0:4] + '_' + data_date_idx[4:]
    ghi_path = file_path + r"/sl_023034_" + ghi_date_idx +'.txt'
    ghi = pd.read_csv (ghi_path) 
    ghi_ori = ghi.copy()

    ghi['timestamp'] = pd.to_datetime(pd.DataFrame ({'year' : ghi['Year Month Day Hours Minutes in YYYY'].values, 
                                                    'month' : ghi['MM'], 
                                                    'day' : ghi['DD'], 
                                                   'hour' : ghi['HH24'], 
                                                   'minute' : ghi['MI format in Local standard time']}))
    ghi.set_index('timestamp', inplace = True)
    # Deal with the space characters (ghi is in object/string form at the moment)
    ghi['Mean global irradiance (over 1 minute) in W/sq m'] = [float(ghi_t) if ghi_t.count(' ')<= 3 else np.nan for ghi_t in ghi['Mean global irradiance (over 1 minute) in W/sq m']]
    
    return data, ghi, data_ori, ghi_ori

C:\Users\samha\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# CHECK ALL DAYS
Find list of all dates which data are available

In [2]:
def create_date_list(year, month, num_of_days):
    """Create a list of date as str for certain month and year.

    Args:
        year (str): year
        month (str): month
        num_of_days (int): number of days for a certain month and year. 

    Returns:
        date_list (list): list of date as str
    """
    
    date_list = []
    
    for i in range(1, 10):
        date_list.append(year + '-' + month + '-0'  + str(i))
    for i in range(10, num_of_days + 1):
        date_list.append(year + '-' + month + '-'  + str(i))
    
    return date_list

In [3]:
datelist_201909 = create_date_list('2019', '09', 30)
datelist_201910 = create_date_list('2019', '10', 31)
datelist_201911 = create_date_list('2019', '11', 30)
datelist_201912 = create_date_list('2019', '12', 31)
datelist_202001 = create_date_list('2020', '01', 31)
datelist_202002 = create_date_list('2020', '02', 29)
datelist_202003 = create_date_list('2020', '03', 31)
datelist_202004 = create_date_list('2020', '04', 30)


date_list = datelist_201909 + datelist_201910 + datelist_201911 + datelist_201912 + datelist_202001 + datelist_202002 + datelist_202003 + datelist_202004

# CLEAR SKY DAYS LIST

In [4]:
csd_201909 = ['2019-09-12', '2019-09-17', '2019-09-18', '2019-09-26', '2019-09-29', '2019-09-30']
csd_201910 = ['2019-10-01', '2019-10-02', '2019-10-03', '2019-10-29']
csd_201911 = ['2019-11-18', '2019-11-27']
csd_201912 = ['2019-12-16', '2019-12-19', '2019-12-25', '2019-12-28']
csd_202001 = ['2020-01-01', '2020-01-02', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-17', '2020-01-29', '2020-01-30']
csd_202002 = ['2020-02-04', '2020-02-05', '2020-02-06', '2020-02-10', '2020-02-11', '2020-02-13', '2020-02-22', '2020-02-23']
csd_202003 = ['2020-03-03', '2020-03-08', '2020-03-10', '2020-03-14', '2020-03-15', '2020-03-25', '2020-03-26']
csd_202004 = ['2020-04-08', '2020-04-09', '2020-04-13', '2020-04-24']

csd_list = csd_201909 + csd_201910 + csd_201911 + csd_201912 + csd_202001 + csd_202002 + csd_202003 + csd_202004

# INPUT ALL MONTHLY DATA & GENERAL FILES

In [ ]:
#file_path = r"/Users/samhan/Downloads/data" #for running in Samhan's laptop
file_path = r"C:\Users\samha\Documents\CANVAS\data" #for running in TETB CEEM09 computer

month_list = ['201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004']
data = {}
data_ori = {}
ghi = {}
ghi_ori = {}
for month in month_list:
    print('processing {} ...'.format(month))
    data[month], ghi[month], data_ori[month], ghi_ori[month] = input_monthly_files(file_path, month)
    data[month]['date'] = data[month].index.date
    
site_details, unique_cids= file_processing.input_general_files(file_path)

processing 201909 ...
processing 201910 ...
processing 201911 ...


# LOOP OVER ALL SITES

In [ ]:
summary = pd.DataFrame()

for c_id in unique_cids_7['c_id']: #for the whole data testing
# for c_id in unique_cids['c_id'][:3]: #for testing
# for c_id in [1317822057]: #for testing
    last_month_analyzed = 0
    count_curtailment = 0
    total_energy_generation = 0
    total_tripping_curtailment = 0
    for date in date_list: 
        print('Analyzing c_id {} date {} ...'.format(c_id, date))
        
        #filter D-PV time series data
        date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()
        month_analyzed = date[:4] + date [5:7]
        
        if not month_analyzed == last_month_analyzed:
            site_id_filter = data[month_analyzed]['c_id'] == c_id
            data_certain_site = data[month_analyzed].loc[site_id_filter]
            last_month_analyzed = month_analyzed
            
        date_filter = data_certain_site['date'] == date_dt
        data_analyzed = data_certain_site.loc[date_filter]

        #check tripping curtailment
        size_is_ok = file_processing.check_data_size(data_analyzed)
        if not size_is_ok:
            print('Cannot analyze {} site on {} due to incomplete data.'.format(c_id, date))
        else:
            data_site, ac_cap, dc_cap, EFF_SYSTEM, inverter = vvar_curt.site_organize(c_id, site_details, data_analyzed, unique_cids)
            data_site = file_processing.resample_in_minute(data_site)
            
            if date in csd_list:
                is_clear_sky_day = True
            else:
                is_clear_sky_day = False
            
            tripping_response, tripping_curt_energy, estimation_method, data_site = tripping_curt.check_tripping_curtailment(is_clear_sky_day, c_id, data_site, unique_cids, ac_cap, site_details, date)    
            energy_generated, data_site = energy_calculation.check_energy_generated(data_site, date, is_clear_sky_day, tripping_curt_energy)
            
            total_energy_generation += energy_generated
            
            if tripping_curt_energy > 0:
                count_curtailment += 1
                total_tripping_curtailment += tripping_curt_energy
    
    #summarize result
    c_id_summary = pd.DataFrame({
            'c_id' : [c_id],
            'total_energy_generation (kWh)' : [total_energy_generation],
            'total_tripping_curtailment (kWh)' : [total_tripping_curtailment],
            'count curtailment (days)' : [count_curtailment],
            'total days (days)' : [len(date_list)]
        })
    
    summary = pd.concat([summary, c_id_summary])

In [ ]:
summary

# SUMMARY PROCESSING

In [ ]:
summary['percentage_of_generation_lost'] = summary['total_tripping_curtailment (kWh)'] / summary['total_energy_generation (kWh)'] * 100
summary['proportion of days with curtailment'] = summary['count curtailment (days)'] / summary['total days (days)'] * 100
summary.sort_values(by = 'percentage_of_generation_lost', ascending = False, inplace = True)
summary.reset_index(drop = True, inplace = True)
summary['proportion_of_sites'] = summary.index/(len(summary)-1) * 100

In [ ]:
summary.to_csv(file_path + r'/tripping_alldays_7.csv', index = False) 

# VISUALIZATION SIMILAR TO FIGURE 14

In [ ]:
#file_path = r"/Users/samhan/Downloads/data" #for running in Samhan's laptop
file_path = r"C:\Users\samha\Documents\CANVAS\data" #for running in TETB CEEM09 computer

summary = pd.read_csv(file_path +r'/tripping_alldays_summary_all.csv')

In [ ]:
summary

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(18, 10)

ax1.scatter(summary['proportion_of_sites'], summary["percentage_of_generation_lost"], color = 'b', 
            label = 'Percentage generation lost')

ax1.set_xlabel('Proportion of sites', **fontdict)
ax1.set_ylabel('Percentage of generation lost per site', **fontdict)

ax1.set_xticks([0, 20, 40, 60, 80, 100])
ax1.set_xticklabels(['0 %', '20 %', '40 %', '60 %', '80 %', '100 %'], **fontdict)
ax1.set_ylim(0, 25);
ax1.set_yticks([0, 5, 10, 15, 20, 25])
ax1.set_yticklabels(['0 %', '5 %', '10 %', '15 %', '20 %', '25 %'], **fontdict)
ax1.legend(loc = 9, prop={'size': 15})

ax2 = ax1.twinx()
ax2.scatter(summary['proportion_of_sites'], summary['proportion of days with curtailment'], color='r',
           label = 'Proportion of days with some curtailment')
ax2.set_ylabel('Proportion of days with some curtailment', **fontdict)
ax2.set_yticks([0, 20, 40, 60, 80, 100])
ax2.set_yticklabels(['0 %', '20 %', '40 %', '60 %', '80 %', '100 %'], **fontdict)
ax2.set_ylim(0, 100);
ax2.legend(loc = 0, prop={'size': 15})

plt.show()

# UPDATING SPREADSHEET

In [ ]:
spreadsheet_before = pd.read_excel(file_path + r'/SolA Curtailment Summary_Final_tripping.xlsx')
#spreadsheet_before.drop(columns = 'Unnamed: 0', inplace = True)

new_data = summary[['c_id', 'total_energy_generation (kWh)', 'total_tripping_curtailment (kWh)']].copy()
new_data.rename(columns={
    "total_energy_generation (kWh)": "total_energy_generation_alldays (kWh)",
    "total_tripping_curtailment (kWh)": "total_tripping_curtailment_alldays (kWh)"}, inplace = True)

spreadsheet_after = pd.merge(spreadsheet_before, new_data, left_on = 'c_id', right_on = 'c_id', how = 'inner')

In [ ]:
spreadsheet_after

In [ ]:
spreadsheet_after.to_excel(file_path + r'/SolA Curtailment Summary_Final_trippingalldays.xlsx', index = False) 

# ANALYZE A SITE FOR A CERTAIN DAY

Just for testing purpose. It is not used in the actual script deployment

In [ ]:
c_id = 814682996
date = '2020-04-23'

In [ ]:
print('Analyzing c_id {} date {} ...'.format(c_id, date))
        
#filter D-PV time series data
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()
month_analyzed = date[:4] + date [5:7]

date_filter = data[month_analyzed].date == date_dt
site_id_filter = data[month_analyzed]['c_id'] == c_id
data_analyzed = data[month_analyzed].loc[date_filter & site_id_filter]

#check tripping curtailment
size_is_ok = file_processing.check_data_size(data_analyzed)
if not size_is_ok:
    print('Cannot analyze {} site on {} due to incomplete data.'.format(c_id, date))
else:
    data_site, ac_cap, dc_cap, EFF_SYSTEM, inverter = vvar_curt.site_organize(c_id, site_details, data_analyzed, unique_cids)
    data_site = file_processing.resample_in_minute(data_site)

In [ ]:
data_site['power'].plot()

In [ ]:
# data_site, polyfit, is_good_polyfit_quality = polyfit_f.check_polyfit(data_site, ac_cap)
if date in csd_list:
    is_clear_sky_day = True
else:
    is_clear_sky_day = False    
tripping_response, tripping_curt_energy, estimation_method, data_site = tripping_curt.check_tripping_curtailment(is_clear_sky_day, c_id, data_site, unique_cids, ac_cap, site_details, date)    
energy_generated, data_site = energy_calculation.check_energy_generated(data_site, date, is_clear_sky_day, tripping_curt_energy)



In [ ]:
energy_generated

In [ ]:
tripping_response

In [ ]:
tripping_curt_energy

In [ ]:
estimation_method

In [ ]:
data_analyzed['power'].plot()

In [ ]:
tripping_response

In [ ]:
energy_generated

In [ ]:
data_site

In [ ]:
site_details.loc[site_details['c_id'] == 358283166 ]

In [ ]:
data_analyzed['power'].plot()

In [ ]:
(data_analyzed['power'].abs() > 300).any()

In [ ]:
data_analyzed.sort_index()

In [ ]:
unique_cids.loc[unique_cids['c_id'] == 814682996]

In [ ]:
unique_cids_4 = unique_cids.loc[147:198]

In [ ]:
unique_cids_5 = unique_cids.loc[198:301]
unique_cids_6 = unique_cids.loc[301:401]
unique_cids_7 = unique_cids.loc[401:]

In [ ]:
unique_cids_4

# COMBINE ALL RESULTS

This is to combine some of the results in the previous section that is done in seperate times. Should not be needed if the above section is run only once for all sites. 

In [ ]:
summary_1 = pd.read_csv(file_path + r'/tripping_alldays_1.csv')
summary_2 = pd.read_csv(file_path + r'/tripping_alldays_2.csv')
summary_3 = pd.read_csv(file_path + r'/tripping_alldays_3.csv')
summary_4 = pd.read_csv(file_path + r'/tripping_alldays_4.csv')
summary_5 = pd.read_csv(file_path + r'/tripping_alldays_5.csv')
summary_6 = pd.read_csv(file_path + r'/tripping_alldays_6.csv')
summary_7 = pd.read_csv(file_path + r'/tripping_alldays_7.csv')

In [ ]:
summary_all_sites = pd.concat([summary_1, summary_2, summary_3, summary_4, summary_5, summary_6, summary_7])

In [ ]:
summary_all_sites.sort_values(by = 'percentage_of_generation_lost', ascending = False, inplace = True)
summary_all_sites.reset_index(drop = True, inplace = True)
summary_all_sites['proportion_of_sites'] = summary_all_sites.index/(len(summary_all_sites)-1) * 100

In [ ]:
summary_all_sites.to_csv(file_path + r'/tripping_alldays_summary_all.csv', index = False) 

# MESSY BELOW